In [1]:
import urllib.request
import json
import ssl
import pandas as pd
from bokeh.plotting import figure,ColumnDataSource
from bokeh.models import HoverTool,WMTSTileSource,LinearColorMapper,LabelSet
from bokeh.palettes import RdPu9 as palette
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.tile_providers import CARTODBPOSITRON_RETINA
from bokeh.application.handlers.function import FunctionHandler
import numpy as np
#HEADER
ssl._create_default_https_context = ssl._create_unverified_context
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]

# COORDINATE CONVERSION FUNCTION
def wgs84_to_web_mercator(df, lon="Long", lat="Lat"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

#FLIGHT TRACKING FUNCTION
def flight_track(doc):
    # initiate bokeh column data source
    flight_source = ColumnDataSource({'Long':[],'Lat':[],'Op':[],'From':[],'To':[],'GAlt':[],'x':[],'y':[],'Call':[]})
    
    
    # UPDATING FLIGHT DATA
    def update():
        #SEND REQUEST, READ RESPONSE AND LOAD AS JSON
        fp=opener.open('http://public-api.adsbexchange.com/VirtualRadar/AircraftList.json?lat=52.379189&lng=4.899431&fDstL=0&fDstU=400')
        mybyte=fp.read()
        mystr=mybyte.decode("utf8")
        js_str=json.loads(mystr)
        fp.close()
        
        #CONVERT TO PANDAS DATAFRAME
        flight_data=js_str['acList']
        flight_df=pd.DataFrame(flight_data,columns=['Long','Lat','Op','From','To','GAlt','Call'])
        wgs84_to_web_mercator(flight_df)
        flight_df=flight_df.fillna('No Data')
        
        # CONVERT TO BOKEH DATASOURCE AND STREAMING
        n_roll=len(flight_df.index)
        flight_source.stream(flight_df.to_dict(orient='list'),n_roll)
        
    #CALLBACK UPATE IN AN INTERVAL
    doc.add_periodic_callback(update, 1000)
    
    #PLOT THE AIRCRAFT POSITION WITH BOKEH
    x_range,y_range=([150000, 500000], [6500000, 7000000]) #bounding box for east US
    p=figure(x_range=x_range,y_range=y_range,x_axis_type='mercator',y_axis_type='mercator',sizing_mode='scale_width',plot_height=300)
    my_hover=HoverTool()
    color_mapper = LinearColorMapper(palette=palette)
    my_hover.tooltips=[('Op','@Op'),('From','@From'),('To','@To'),('GAlt','@GAlt')]
    labels = LabelSet(x='x', y='y', text='Call', level='glyph',
            x_offset=5, y_offset=5, source=flight_source, render_mode='canvas',background_fill_color='#943390',text_font_size="12pt",border_line_alpha=1.0, text_color='#ffffff')
    p.add_tile(CARTODBPOSITRON_RETINA)
    p.circle('x','y',source=flight_source,fill_color={'field': 'GAlt', 'transform': color_mapper},hover_color='yellow',size=15,fill_alpha=0.8,line_width=0.5)
    p.add_tools(my_hover)
    p.add_layout(labels)
    doc.title='REAL TIME FLIGHT TRACKING WITH PANDAS AND BOKEH'
    doc.add_root(p)
    
# SERVER CODE
apps = {'/': Application(FunctionHandler(flight_track))}
server = Server(apps, port=5000) #define an unused port
server.start()